In [1]:
import os
import pickle
import torch
import exp_configs

In [24]:
import wandb

#entity = 'cnn'
#project = 'multilabel_ar_hmdb_202207'
entity = 'kiyoon'
project = 'multilabel_ar_feature_20220519'
api = wandb.Api()

runs = api.runs(f"{entity}/{project}")

In [27]:
# find the run

#dataset = 'confusing_hmdb_102_features'
dataset = 'epic100_verb_features'
loss = 'concat_RGB_flow_assume_negative'
#loss = 'concat_RGB_flow_entropy_maximise'
#split = 3
seed = 13
#search_run_names = [f'{dataset} ch_beta.featuremodel {loss} split{split}']
search_run_names = [f'{dataset} ch_beta.featuremodel {loss} seed{seed}']
#search_run_names = ['hmdb_confusion2 tsm_resnet50_nopartialbn entropy_maximise split3', 'hmdb_confusion2 ch_epic100.tsm_resnet50_flow entropy_maximise split3']
found_runs = []
for run in runs:
    for search_run_name in search_run_names:
        if search_run_name in run.name:
            print(run.name, run.id)
            found_runs.append(run)

assert len(found_runs) == len(search_run_names), f'{len(found_runs) = }'

predictions = []
for run in found_runs:
    # find artifact from the run.
    produced_artifacts = run.logged_artifacts()
    artifact_name = None
    for art in produced_artifacts:
        if art.name.startswith('predictions'):
            print(art.name)
            artifact_name = art.name

    # Download the artifact
    artifact = api.artifact(f"{entity}/{project}/{artifact_name}")
    artifact_dir = artifact.checkout()

    # open
    best_epoch = run.summary['epoch_best-val_epoch']
    with open(os.path.join(artifact_dir, f'epoch_{best_epoch:04d}_val_oneclip.pkl'), 'rb') as f:
        data = pickle.load(f)
    predictions.append(data)

epic100_verb_features ch_beta.featuremodel concat_RGB_flow_assume_negative seed13 v0 1vtg2xlm
predictions:v0


In [28]:
# Compute sigmoid average of two predictions
sigmoid_1 = torch.sigmoid(torch.tensor(predictions[0]['video_predictions']))

In [29]:
sigmoid_1.mean()

tensor(0.5025)

In [233]:
sigmoid_2

tensor([[9.5276e-01, 1.0402e-05, 6.3911e-06,  ..., 5.7971e-06, 1.1700e-06,
         1.5672e-03],
        [9.8327e-01, 6.8464e-07, 5.9809e-06,  ..., 2.3575e-07, 1.8863e-08,
         2.9382e-03],
        [9.7815e-01, 4.8019e-06, 1.2909e-05,  ..., 7.8160e-06, 2.1755e-05,
         5.8592e-04],
        ...,
        [1.0262e-03, 1.1863e-05, 2.2659e-03,  ..., 3.1138e-03, 1.6888e-04,
         3.6149e-02],
        [6.4585e-06, 9.8888e-07, 5.7388e-04,  ..., 5.4901e-03, 2.3314e-05,
         4.9621e-02],
        [1.6512e-02, 1.2595e-07, 2.4703e-05,  ..., 2.2985e-03, 6.0593e-05,
         6.6771e-03]])

In [234]:
sigmoid_average

tensor([[0.9597, 0.2314, 0.2342,  ..., 0.2464, 0.2348, 0.2751],
        [0.9812, 0.2367, 0.2396,  ..., 0.2586, 0.2454, 0.2983],
        [0.9697, 0.2401, 0.2405,  ..., 0.2787, 0.2597, 0.2798],
        ...,
        [0.2622, 0.2284, 0.2572,  ..., 0.3235, 0.3317, 0.2760],
        [0.2211, 0.2351, 0.2625,  ..., 0.2832, 0.2446, 0.3065],
        [0.3411, 0.2631, 0.2853,  ..., 0.2343, 0.2441, 0.2975]])

In [235]:
# calculate metrics
cfg = exp_configs.load_cfg('hmdb_confusion2', 'tsm_resnet50_nopartialbn', 'assume_negative', dataset_channel='beta', exp_channel='labelsmth')
print(cfg.metrics['val'])
for metric in cfg.metrics['val']:
    metric.clean_data()
    metric.add_clip_predictions(torch.tensor(predictions[0]['video_ids']), sigmoid_average, torch.tensor(predictions[0]['video_labels']))
    metric.calculate_metrics()
    print(metric.last_calculated_metrics)

[<pyvideoai.metrics.multilabel_accuracy.ClipMultilabelAccuracyMetric object at 0x7f4f9b164af0>, <pyvideoai.metrics.top1_multilabel_accuracy.ClipTop1MultilabelAccuracyMetric object at 0x7f4f9b164760>, <pyvideoai.metrics.top1_multilabel_accuracy.ClipTopkMultilabelAccuracyMetric object at 0x7f4f9b164730>, <pyvideoai.metrics.mAP.Clip_mAPMetric object at 0x7f4f9b164700>]
0.592156862745098
0.6104575163398693
0.6928104575163399
0.6187192633711707
